# Visualising FITS and star positions

## Import modules

In [156]:
import requests, math, glob
import numpy as np
from astropy.stats import mad_std
from astropy.io import fits
from astropy.wcs import WCS
import astropy.units as u
from astropy.coordinates import Angle
import matplotlib.pyplot as plt
from photutils import aperture_photometry, CircularAperture
from pathlib import Path
warnings.filterwarnings('ignore')

import os
os.chdir('/home/jovyan/work/src')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Set reference frame, global vars

In [178]:
print(os.getcwd())
fits_file = Path('../notebooks/data/V451Dra#15C_000000246.FIT')
FITS_FILE= wcs_file



/home/jovyan/work/src


## Define binning functions

In [172]:
from skimage.measure import block_reduce
def binning_fast(arr, binning_arr=(2,2)):
    return block_reduce(arr, block_size=binning_arr, func=np.sum)

def adjust_header(header):
    header['XPIXSZ'] = header['XPIXSZ']*2
    header['YPIXSZ'] = header['YPIXSZ']*2
    header['XBINNING'] = header['XBINNING']*2
    header['YBINNING'] = header['YBINNING']*2

def binned_filename(fits):
    return fits.parent/(fits.stem+'_binned'+fits.suffix)

In [173]:
def getWcs(wcs_file):
    hdulist = fits.open(wcs_file)
    data = hdulist[0].data.astype(float)
    header = hdulist[0].header
    wcs = WCS(header)
    return wcs

wcs = getWcs(wcs_file)



In [174]:
print(wcs)
wcs.naxis

WCS Keywords

Number of WCS axes: 2
CTYPE : ''  ''  
CRVAL : 0.0  0.0  
CRPIX : 0.0  0.0  
PC1_1 PC1_2  : 1.0  0.0  
PC2_1 PC2_2  : 0.0  1.0  
CDELT : 1.0  1.0  
NAXIS : 4788  3194


2

## Get FIT

In [175]:
print(FITS_FILE)
hdulist = fits.open(FITS_FILE)
# data = hdulist[0].data.astype(float)
data = hdulist[0].data

../notebooks/data/V451Dra#15C_000000246.FIT


In [191]:
print(data.shape)

fig=plt.figure(figsize=(36, 32), dpi= 80, facecolor='w', edgecolor='k')
# target_app = CircularAperture(target_xy, r=20.)    
#plt.imshow(data, cmap='gray_r', origin='lower', vmin=0, vmax=2500)
%time data_binned=binning_fast(data)
hdulist[0].data = data_binned.astype(np.uint16)
#hdulist[0].header = adjust_header(hdulist[0].header)
adjust_header(hdulist[0].header)
#print(hdulist[0].header.__repr__)
#plt.imshow(data_binned, cmap='gray_r', origin='lower')
print(type(hdulist[0].data[0][0]))

(3194, 4788)
CPU times: user 129 ms, sys: 116 ms, total: 245 ms
Wall time: 250 ms
<class 'numpy.uint16'>


<Figure size 2880x2560 with 0 Axes>

## Save the image

In [192]:
#print((Path(wcs_file.parent/wcs_file.stem)+'_binned.fits'+wcs_file.suffix))

hdulist.writeto(binned_filename(fits_file), overwrite=True)
plt.close(fig)

In [76]:
wcs_file.parent/wcs_file.stem+Path('bla')

TypeError: unsupported operand type(s) for +: 'PosixPath' and 'PosixPath'

In [111]:
print(a)
%time result =binning(a)
print(result)
%time result = binning_fast(a)

[[ 0  1  2  3  4  5]
 [ 6  7  8  9 10 11]
 [12 13 14 15 16 17]
 [18 19 20 21 22 23]
 [24 25 26 27 28 29]
 [30 31 32 33 34 35]]
m,n 6 6
Input is (6, 6), output is (3, 3)
CPU times: user 0 ns, sys: 1.34 ms, total: 1.34 ms
Wall time: 1.1 ms
[[ 14.  22.  30.]
 [ 62.  70.  78.]
 [110. 118. 126.]]
CPU times: user 0 ns, sys: 798 µs, total: 798 µs
Wall time: 795 µs


In [193]:
hdulist[0].header

SIMPLE  =                    T                                                  
BITPIX  =                   16 / 8 unsigned int, 16 & 32 int, -32 & -64 real    
NAXIS   =                    2 /number of axes                                  
NAXIS1  =                 2394 / fastest changing axis                          
NAXIS2  =                 1597 / next to fastest changing axis                  
DATE-OBS= '2020-02-06T23:06:31' /YYYY-MM-DDThh:mm:ss observation start, UT      
EXPTIME =   15.000000000000000 /Exposure time in seconds                        
EXPOSURE=   15.000000000000000 /Exposure time in seconds                        
SET-TEMP=  -10.000000000000000 /CCD temperature setpoint in C                   
CCD-TEMP=  -10.000000000000000 /CCD temperature at start of exposure in C       
XPIXSZ  =              7700.48 / Pixel Width in microns (after binning)         
YPIXSZ  =              7700.48 / Pixel Height in microns (after binning)        
XBINNING=                 20

In [170]:
print(wcs_file.parent/(wcs_file.stem+'_binned'+wcs_file.suffix))

../notebooks/data/V451Dra#15C_000000246_binned.FIT


In [168]:
wcs_file.stem+'_binned'+wcs_file.suffix

'V451Dra#15C_000000246_binned.FIT'